# User-Based Collaborative Filtering with Word2Vec

This notebook can be used to train and test user-based collaborative filtering models using Word2Vec. Before you start, a couple things are needed:

1. A database with scraped movie reviews and movie data. See the Groa/web_scraping folder for a [scraper](https://github.com/Lambda-School-Labs/Groa/blob/master/web_scraping/scraper.py) you can use to get the reviews. Movie data can be found [here](https://datasets.imdbws.com/). You'll want the files 'title.basics.tsv.gz' and 'title.ratings.tsv.gz'.

2. Folders in the current working directory titled:
    - /models
        - This is where trained models are saved.
    - /training_data
        - This is where training data is saved.
    - /exported_data
         - This contains a small version of the IMDb title.basics.tsv file that we use to quickly retrieve movie info.
         - It also contains /letterboxd and /imdb, where test data is saved for scoring various models. I have included my own data as an example of how to format any test data you can collect. The letterboxd data is just the unzipped Letterboxd export folder, exactly as it comes from the site.


## Connect to Database
Do this at the start of every session.

In [1]:
! pip3 install psycopg2-binary --user
import pandas as pd
import psycopg2
import numpy as np
from getpass import getpass

# connect to database. Never save your password to a notebook.
connection = psycopg2.connect(
    database  = "postgres",
    user      = "postgres",
    password  = getpass(), # secure password entry. Enter DB password in the prompt and press Enter.
    host      = "movie-rec-scrape.cvslmiksgnix.us-east-1.rds.amazonaws.com",
    port      = '5432'
)

# create cursor that is used throughout
try:
    c = connection.cursor()
    print("Connected!")
except Exception as e:
    print("Connection problem chief!\n")
    print(e)

You should consider upgrading via the 'c:\users\cooper\anaconda3\python.exe -m pip install --upgrade pip' command.


 ················


Connected!


## Data preparation plan:

1. Get the list of reviewers whose reviews we want (about 17k).
    - We only want to train on users who have positively rated a minimum number of movies, otherwise their watch histories will give us poor-quality associations between movies. So we set the minimum number as the variable `n` below.
2. Create the dataframe of reviewers, movie IDs with positive reviews
3. Inner join the above two dataframes to remove positive reviews whose reviewers don't meet our criteria.
4. Run the list constructor on the join table to construct the training data.
    - Training data is of this format: [['movieid1', 'movieid2', ...],['movieid3', 'movieid4', ...], ...]
                                      ^user1 watch history     ^user2 watch history 
5. Save the training data.
6. Train Word2Vec on the list of watch histories (which are themselves lists of movie IDs).

In [84]:
# Get reviewers with at least n positive reviews (rating m-10 inclusive)

n = 5 # minimum number of positive reviews for a watch history to be included
m = 7 # minimum star rating for inclusion (IMDb rating scale)

c.execute(f"""
SELECT username
FROM reviews
WHERE user_rating BETWEEN {m} AND  10
GROUP BY username
HAVING COUNT(username) >= {n}
ORDER BY COUNT(username) DESC
""")

'''
Minimum rating for training data has been increased to 8 stars in v3_LimitingFactor.

Explanation: v2_MistakeNot is returning movies with an average rating of 7.66, 
which is towards the low end of the distribution in the training data. It might be 
near the mean, but we want our model to give the user an above-average movie experience.


'''
reviewers = c.fetchall()

In [85]:
# how many reviewers qualify for inclusion?
len(reviewers)

44438

#### Training note: 

The following query currently returns reviews in no discernible order.
This is because the reviews were inserted into the database by multiple scrapers
running in parallel.

Future users of this notebook should take care to note whether their database gives
the same result. The reason the order is important is that Word2Vec trains by learning to predict the movies
within a 10-movie window. A non-random ordering may introduce a bias. This bias might improve the model, e.g. 
in the case where training data is sorted by review date. For all our initial models, however, we have elected
not to pursue that approach. 

There are two reasons for this:
1. Ultimately, the user "taste vector" that is the input for making recommendations is
a vector average of all the movies the user has watched, so it's not perfectly analogous to finding similar movies
to a single movie.
2. More importantly, sorting the training data by review date would influence the model to associate movies
according to the order people watch them in. This has pros and cons. We don't want to provide recommendations 
that lead a user down a path that others have trodden, and this would seem to be one potential drawback. But further
testing is needed. It might be a great idea.

In [93]:
# Get IDs of all positive reviews from database with minimum star rating m
# Note: we didn't scrape Letterboxd reviews so we only have 10-scale ratings

# query for IDs
c.execute(f"""SELECT movie_id, username 
            FROM reviews 
            WHERE user_rating BETWEEN {m} and 10
            ORDER BY review_date""")
result = c.fetchall()

# create reviews dataframe
df = pd.DataFrame(result, columns = ['movieid', 'userid'])
df.head()

,movieid,userid
0,0070047,robh
1,0120201,robh
2,0084787,robh
3,0120746,jreeves
4,0117571,robh


In [94]:
# create reviewers dataframe
df_reviewers = pd.DataFrame(reviewers, columns = ['userid'])

In [95]:
# number of reviewers
df_reviewers.shape

(44438, 1)

In [96]:
# merge to get only the IDs relevant to training
df = df.merge(df_reviewers, how='inner', on='userid')
df.shape

(986835, 2)

In [97]:
# ! sudo su
# ! yum update -y
# ! yum -y install python-pip
! python -V #should be python 3.7

Python 3.7.3


In [98]:
# ! which pip

### Install gensim

In [99]:
! python -m pip install tqdm # this is just a terminal loading bar, not necessary
# ! python -c 'import tqdm'
! python -m pip install gensim

ERROR: Invalid requirement: '#'
You should consider upgrading via the 'C:\Users\Cooper\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Cooper\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [101]:
# list to capture watch history of the users
watched_train = []

# populate the list with the movie codes
for i in reviewers:
    temp = df[df["userid"] == i[0]]["movieid"].tolist()
    watched_train.append(temp)
    
len(watched_train) # number of watch histories

44438

In [102]:
# save the data so we don't lose all that hard work
import pickle
pickle.dump(watched_train, open(f'training_data/watched_train_{m}-10_{n}reviews_ordered.sav', 'wb'))

In [166]:
# # uncomment if you want to save the model in protocol 2 so it can be opened in python 2.7
# import pickle
# temp = pickle.load(open('watched_train.sav', 'rb'))
# pickle.dump(temp, open('watched_train.sav', 'wb'), protocol=2)

## Train the Model

Small note:
The first version of this model was trained on movie IDs that were inside lists of length 1, with watch histories being lists of lists. This version eschews the inner lists. Each watch history is simply a list of strings.

In [ ]:
# should be 3.6.0 or above
gensim.__version__

In [ ]:
# load training data
import pickle
watched_train = pickle.load(open(f'training_data/watched_train_{m}-10_{n}reviews_ordered.sav', 'rb'))
len(watched_train)

### Tuning Hyperparameters

Much insight can be gained from reading [Word2vec applied to Recommendation: Hyperparameters Matter](https://www.groundai.com/project/word2vec-applied-to-recommendation-hyperparameters-matter/1).

- 4 hyperparameters that can significantly improve results:
    - negative sampling distribution
        - not included yet.
        - (negative) 0.5 in best results
    - number of epochs
        - 90-150 in the best results.
    - subsampling parameter
        - 10^-4 in all the best results.
    - window-size
        - Set to 6000 to capture whole history?
        - Perhaps it shouldn't be so large, to capture movies watched around the same time. But that requires ordering the training data by review date. Great experiment to try. 
        - Best results set this 3-7. Surprisingly small.
- `Tuning the parameters seems to drive the algorithm towards organizing the space in a way or another (e.g. better positioning the top items in the space, or pushing away less demanded items). Furthermore, the homogeneity of popularity between items of a same sequence, the shape of the popularity distribution, or the heterogeneity of the items in the catalog have a direct impact on the task evaluation.`
    - Probably do this after the first try at optimizing the above.
- `Some authors claim without empirical or theoretical verification that it is best to use a ”infinite” window-size (Barkan and
Koenigstein, [2016](https://www.groundai.com/project/word2vec-applied-to-recommendation-hyperparameters-matter/1#bib.bib2)), meaning that the whole sessions is considered as one context, but most arbitrarily used a fixed value without further discussion.`


In [103]:
import random
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings;
warnings.filterwarnings('ignore')

# train word2vec model
model = Word2Vec(
                 size = 100, # vector size
                 window = 10, # perhaps increase this
                 sg = 1, # sets to skip-gram
                 hs = 0, # must be set to 0 for negative sampling
                 negative = 10, # for negative sampling
                 ns_exponent = 0.3, # 0.5 in best results
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14,
                 sample = 0.0001 # 10^-4 in best results
                )
model.min_rating = m # store the minimum rating of training data
model.min_pos_reviews = n # store the minimum number of positive reviews
model.training_order = "review_date"
model.build_vocab(watched_train, progress_per=200)
model.train(watched_train, total_examples = model.corpus_count, 
            epochs=1500, # best results set this 90-150
            report_delay=60, compute_loss=True)

# save word2vec model
model.save("models/w2v_limitingfactor_v3.36.model")

## Use the model

In [ ]:
# !pip install gensim==3.8.1

### Define inferencing functions

In [85]:
import gensim
import pandas as pd
import re
import warnings;
warnings.filterwarnings('ignore')
    
def fill_id(id):
    """Adds leading zeroes back if necessary. This makes the id match the database."""
    if len(str(id)) < 7:
        length = len(str(id))
        id = "0"*(7 - length) + str(id)
    return str(id)
    
def df_to_id_list(df, id_book):
    """Converts dataframe of movies to a list of the IDs for those movies.

    Every title in the input dataframe is checked against the local file, which
    includes all the titles and IDs in our database. For anything without a match,
    replace the non-alphanumeric characters with wildcards, and query the database
    for matches.
    """
    df['Year'] = df['Year'].astype(int).astype(str)
    matched = pd.merge(df, id_book,
                left_on=['Name', 'Year'], right_on=['primaryTitle', 'startYear'],
                how='inner')
    ids = matched['tconst'].astype(str).tolist()
    final_ratings = []
    names = df.Name.tolist()
    years = [int(year) for year in df.Year.tolist()]
    if 'Rating' in df.columns:
        stars = [int(rating) for rating in df.Rating.tolist()]
        info = list(zip(names, years, stars))
        final_ratings = matched['Rating'].astype(int).tolist()
    else:
        info = list(zip(names, years, list(range(len(years)))))
    missed = [x for x in info if x[0] not in matched['primaryTitle'].tolist()]
    for i, j, k in missed:
        i = re.sub('[^\s0-9a-zA-Z\s]+', '%', i)
        try:
            cursor_dog.execute(f"""
                SELECT movie_id, original_title, primary_title
                FROM movies
                WHERE primary_title ILIKE '{i}' AND start_year = {j}
                  OR original_title ILIKE '{i}' AND start_year = {j}
                ORDER BY runtime_minutes DESC
                LIMIT 1""")
            id = cursor_dog.fetchone()[0]
            ids.append(id)
            final_ratings.append(k)
        except:
            continue
    ids = [fill_id(id) for id in ids]
    final_ratings = [x*2 for x in final_ratings]
    ratings_dict = dict(zip(ids, final_ratings))
    return tuple([ids, ratings_dict])
    
def prep_data(ratings_df, watched_df=None, watchlist_df=None, 
                   good_threshold=4, bad_threshold=3):
    """Converts dataframes of exported Letterboxd data to lists of movie_ids.

    Parameters
    ----------
    ratings_df : pd dataframe
        Letterboxd ratings.

    watched_df : pd dataframe
        Letterboxd watch history.

    watchlist_df : pd dataframe
        Letterboxd list of movies the user wants to watch.
        Used in val_list for scoring the model's performance.

    good_threshold : int
        Minimum star rating (10pt scale) for a movie to be considered "enjoyed" by the user.

    bad_threshold : int
        Maximum star rating (10pt scale) for a movie to be considered "disliked" by the user.


    Returns
    -------
    tuple of lists of ids.
        (good_list, bad_list, hist_list, val_list)
    """
    try:
        # try to read Letterboxd user data
        # drop rows with nulls in the columns we use
        ratings_df = ratings_df.dropna(axis=0, subset=['Rating', 'Name', 'Year'])
        # split according to user rating
        good_df = ratings_df[ratings_df['Rating'] >= good_threshold]
        bad_df = ratings_df[ratings_df['Rating'] <= bad_threshold]
        neutral_df = ratings_df[(ratings_df['Rating'] > bad_threshold) & (ratings_df['Rating'] < good_threshold)]
        # convert dataframes to lists
        good_list, good_dict = df_to_id_list(good_df, id_book)
        bad_list, bad_dict = df_to_id_list(bad_df, id_book)
        neutral_list, neutral_dict = df_to_id_list(neutral_df, id_book)
    except KeyError:
        # Try to read IMDb user data
        # strip ids of "tt" prefix
        ratings_df['movie_id'] = ratings_df['Const'].str.lstrip("tt")
        # drop rows with nulls in the columns we use
        ratings_df = ratings_df.dropna(axis=0, subset=['Your Rating', 'Year'])
        # split according to user rating
        good_df = ratings_df[ratings_df['Your Rating'] >= good_threshold*2]
        bad_df = ratings_df[ratings_df['Your Rating'] <= bad_threshold*2]
        neutral_df = ratings_df[(ratings_df['Your Rating'] > bad_threshold*2) & (ratings_df['Your Rating'] < good_threshold*2)]
        # convert dataframes to lists
        good_list = good_df['movie_id'].to_list()
        bad_list = bad_df['movie_id'].to_list()
        neutral_list = neutral_df['movie_id'].to_list()
    except Exception as e:
        # can't read the dataframe as Letterboxd or IMDb user data
        print("This dataframe has columns:", ratings_df.columns)
        raise Exception(e)
        
    ratings_dict = dict(list(good_dict.items()) + list(bad_dict.items()) + list(neutral_dict.items()))

    if watched_df is not None:
        # Construct list of watched movies that aren't rated "good" or "bad"
        # First, get a set of identified IDs.
        rated_names = set(good_df.Name.tolist() + bad_df.Name.tolist() + neutral_list)
        # drop nulls from watched dataframe
        full_history = watched_df.dropna(axis=0, subset=['Name', 'Year'])
        # get list of watched movies that haven't been rated
        hist_list = df_to_id_list(full_history[~full_history['Name'].isin(rated_names)], id_book)[0]
        # add back list of "neutral" movies (whose IDs we already found before)
        hist_list = hist_list + neutral_list
    else: hist_list = neutral_list

    if watchlist_df is not None:
        try:
            watchlist_df = watchlist_df.dropna(axis=0, subset=['Name', 'Year'])
            val_list = df_to_id_list(watchlist_df, id_book)[0]
        except KeyError:
            watchlist_df = watchlist_df.dropna(axis=0, subset=['Const', 'Year'])
            watchlist_df['movie_id'] = watchlist_df['Const'].str.lstrip("tt")
            val_list = watchlist_df['movie_id'].tolist()
    else: val_list = []

    return (good_list, bad_list, hist_list, val_list, ratings_dict)

class Recommender(object):

    def __init__(self, model_path):
        """Initialize model with name of .model file"""
        self.model_path = model_path
        self.model = None
        self.cursor_dog = c # set to this notebook's connection
        self.ratings_book = pd.read_csv('exported_data/title_ratings.tsv', delimiter='\t')
        self.ratings_book['id'] = self.ratings_book['movie_id'].str.lstrip('tt').str.lstrip('0').astype(int)                            
        self.id_book = pd.read_csv('exported_data/title_basics_filtered.csv').merge(
                       self.ratings_book, left_on='tconst', right_on='id')

    def connect_db(self):
        """connect to database, create cursor.
        In the notebook, this isn't used, and all connections are 
        handled through the notebook's global connection at the top."""
        # connect to database
        connection = psycopg2.connect(
            database  = "postgres",
            user      = "postgres",
            password  = os.getenv('DB_PASSWORD'),
            host      = "movie-rec-scrape.cvslmiksgnix.us-east-1.rds.amazonaws.com",
            port      = '5432'
        )
        # create cursor that is used throughout

        try:
            self.cursor_dog = connection.cursor()
            print("Connected!")
        except:
            print("Connection problem chief!")

    def _get_model(self):
        """Get the model object for this instance, loading it if it's not already loaded."""
        if self.model == None:
            model_path = self.model_path
            w2v_model = gensim.models.Word2Vec.load(model_path)
            # Keep only the normalized vectors.
            # This saves memory but makes the model untrainable (read-only).
            w2v_model.init_sims(replace=True)
            self.model = w2v_model
        return self.model

    def _get_info(self, id, score=None):
        """Takes an id string and returns the movie info with a url."""
        try:
            info_query = f"""
            SELECT m.primary_title, m.start_year, r.average_rating, r.num_votes
            FROM movies m
            JOIN ratings r ON m.movie_id = r.movie_id
            WHERE m.movie_id = '{id}'"""
            self.cursor_dog.execute(info_query)
        except Exception as e:
            return tuple([f"Movie title unknown. ID:{id}", None, None, None, None, None, id])

        t = self.cursor_dog.fetchone()
        if t:
            title = tuple([t[0], t[1], f"https://www.imdb.com/title/tt{id}/", t[2], t[3], score, id])
            return title
        else:
            return tuple([f"Movie title not retrieved. ID:{id}", None, None, None, None, None, id])

    def get_most_similar_title(self, id, id_list):
        """Get the title of the most similar movie to id from id_list"""
        clf = self._get_model()
        vocab = clf.wv.vocab
        if id not in vocab:
            return ""
        id_list = [id for id in id_list if id in vocab] # ensure all in vocab
        id_book = self.id_book
        match = clf.wv.most_similar_to_given(id, id_list)
        return id_book['primaryTitle'].loc[id_book['tconst'] == int(match)].values[0]

    def predict(self, input, bad_movies=[], hist_list=[], val_list=[],
                ratings_dict = {}, checked_list=[], rejected_list=[],
                n=50, harshness=1, rec_movies=True,
                show_vibes=False, scoring=False, return_scores=False):
        """Returns a list of recommendations and useful metadata, given a pretrained
        word2vec model and a list of movies.

        Parameters
        ----------

            input : iterable
                List of movies that the user likes.

            bad_movies : iterable
                List of movies that the user dislikes.

            hist_list : iterable
                List of movies the user has seen.

            val_list : iterable
                List of movies the user has already indicated interest in.
                Example: https://letterboxd.com/tabula_rasta/watchlist/
                People really load these up over the years, and so they make for 
                the best validation set we can ask for with current resources.

            ratings_dict : dictionary
                Dictionary of movie_id keys, user rating values.

            checked_list : iterable
                List of movies the user likes on the feedback form.

            rejected_list : iterable
                List of movies the user dislikes on the feedback form.

            n : int
                Number of recommendations to return.

            harshness : int
                Weighting to apply to disliked movies.
                Ex:
                    1 - most strongly account for disliked movies.
                    3 - divide "disliked movies" vector by 3.

            rec_movies : boolean
                If False, doesn't return movie recommendations (used for scoring).

            show_vibes : boolean
                If True, prints out the dupes as a feature.
                These movies are closest to the user's taste vector, 
                indicating some combination of importance and popularity.

            scoring : boolean
                If True, prints out the validation score.
            
            return_scores : boolean
                If True, skips printing out

        Returns
        -------
        A list of tuples
            (Title, Year, IMDb URL, Average Rating, Number of Votes, Similarity score)
        """

        clf = self._get_model()
        dupes = []                 # list for storing duplicates for scoring

        def _aggregate_vectors(movies, feedback_list=[]):
            """Gets the vector average of a list of movies."""
            movie_vec = []
            for i in movies:
                try:
                    m_vec = clf[i]  # get the vector for each movie
                    if ratings_dict:
                        try:
                            r = ratings_dict[i] # get user_rating for each movie
                            # Use a polynomial to weight the movie by rating.
                            # This equation is somewhat arbitrary. I just fit a polynomial
                            # to some weights that look good. The effect is to raise
                            # the importance of 1, 2, 9, and 10 star ratings to about 1.8.
                            w = ((r**3)*-0.00143) + ((r**2)*0.0533) + (r*-0.4695) + 2.1867
                            m_vec = m_vec * w
                        except KeyError:
                            continue
                    movie_vec.append(m_vec)
                except KeyError:
                    continue
            if feedback_list:
                for i in feedback_list:
                    try:
                        f_vec = clf[i]
                        movie_vec.append(f_vec*1.8) # weight feedback by changing multiplier here
                    except KeyError:
                        continue
            return np.mean(movie_vec, axis=0)

        def _similar_movies(v, bad_movies=[], n=50):
            """Aggregates movies and finds n vectors with highest cosine similarity."""
            if bad_movies:
                v = _remove_dislikes(bad_movies, v, harshness=harshness)
            self.my_vec = v
            return clf.similar_by_vector(v, topn= n+1)[1:]

        def _remove_dupes(recs, input, bad_movies, hist_list=[], feedback_list=[]):
            """remove any recommended IDs that were in the input list"""
            all_rated = input + bad_movies + hist_list + feedback_list
            nonlocal dupes
            dupes = [x for x in recs if x[0] in input]
            return [x for x in recs if x[0] not in all_rated]

        def _remove_dislikes(bad_movies, good_movies_vec, rejected_list=[], harshness=1):
            """Takes a list of movies that the user dislikes.
            Their embeddings are averaged,
            and subtracted from the input."""
            bad_vec = _aggregate_vectors(bad_movies, rejected_list)
            bad_vec = bad_vec / harshness
            return good_movies_vec - bad_vec

        def _score_model(recs, val_list):
            """Returns the number of recs that were already in the user's watchlist. Validation!"""
            ids = [x[0] for x in recs]
            return len(list(set(ids) & set(val_list)))

        aggregated = _aggregate_vectors(input, checked_list)
        recs = _similar_movies(aggregated, bad_movies, n=n)
        recs = _remove_dupes(recs, input, bad_movies, hist_list, checked_list + rejected_list)
        formatted_recs = [self._get_info(x[0], x[1]) for x in recs]
        if val_list:
            if return_scores:
                return tuple([_score_model(recs, val_list), sum([i[3] for i in formatted_recs if i[3] is not None])/len(formatted_recs)])
            elif scoring:
                print(f"The model recommended {_score_model(recs, val_list)} movies that were on the watchlist!\n")
                print(f"\t\t Average Rating: {sum([i[3] for i in formatted_recs if i[3] is not None])/len(formatted_recs)}\n")
        if show_vibes:
            print("You'll get along with people who like: \n")
            for x in dupes:
                print(self._get_info(x[0], x[1]))
            print('\n')
        if rec_movies:
            return formatted_recs

### Prep generic test data

In [3]:
# df to lookup ids from titles
id_book = pd.read_csv('exported_data/title_basics_small.csv')

# import user Letterboxd data
ratings = pd.read_csv('exported_data/letterboxd/cooper/ratings.csv')
watched = pd.read_csv('exported_data/letterboxd/cooper/watched.csv')
watchlist = pd.read_csv('exported_data/letterboxd/cooper/watchlist.csv')

# note: if you import IMDb data, it's currently encoded 'cp1252' (but they may someday switch to utf-8)

# prep user data
good_list, bad_list, hist_list, val_list, ratings_dict = prep_data(
                                    ratings, watchlist_df=watchlist, good_threshold=4, bad_threshold=3)

In [4]:
print(len(good_list), len(bad_list), len(hist_list), len(val_list))

389 194 146 1062


In [5]:
#-------------------------------------------------------------#
# To inspect a model, enter its path here and run the cell
model_path = "models/w2v_limitingfactor_v3.36.model"
#------------------------------------------------#

model = gensim.models.Word2Vec.load(model_path)

def _get_info(id):
            """Takes an id string and returns the movie info with a url."""
            try:
                c.execute(f"""
                select m.primary_title, m.start_year, r.average_rating, r.num_votes
                from movies m
                join ratings r on m.movie_id = r.movie_id
                where m.movie_id = '{id[0]}'""")
            except:
                return tuple([f"Movie title unknown. ID:{id[0]}", None, None, None, None, None])

            t = c.fetchone()
            if t:
                title = tuple([t[0], t[1], f"https://www.imdb.com/title/tt{id[0]}/", t[2], t[3], id[1]])
                return title
            else:
                return tuple([f"Movie title unknown. ID:{id[0]}", None, None, None, None, None])

print(model)
print(f"""\t
            corpus_count: {model.corpus_count}
            corpus_total_words: {model.corpus_total_words}
            window: {model.window}
            sg: {model.sg}
            hs: {model.hs}
            negative: {model.negative}
            ns_exponent: {model.ns_exponent}
            alpha: {model.alpha}
            min_alpha: {model.min_alpha}
            sample: {model.sample}
            epochs: {model.epochs}
            """)
print("Most similar movies to Porco Rosso minus Kiki's Delivery Service?")
movies = model.similar_by_vector(model['0104652']
                                -model['0097814']
                                 ,topn=11)
for i in movies:
    print(_get_info(i))

Word2Vec(vocab=24691, size=100, alpha=0.03)
	
            corpus_count: 35371
            corpus_total_words: 933359
            window: 10
            sg: 1
            hs: 0
            negative: 10
            ns_exponent: 0.75
            alpha: 0.03
            min_alpha: 0.0007
            sample: 0.0001
            epochs: 1500
            
Most similar movies to Porco Rosso minus Kiki's Delivery Service?
('Porco Rosso', 1992, 'https://www.imdb.com/title/tt0104652/', 7.8, 64422, 0.5604802370071411)
('Kaleidoscope', 1966, 'https://www.imdb.com/title/tt0060581/', 6.0, 613, 0.41448774933815)
('Capote', 2005, 'https://www.imdb.com/title/tt0379725/', 7.3, 119883, 0.39984023571014404)
('Uncertainty', 2008, 'https://www.imdb.com/title/tt1086216/', 5.8, 6290, 0.39574727416038513)
("Von Ryan's Express", 1965, 'https://www.imdb.com/title/tt0059885/', 7.1, 12038, 0.3899157643318176)
('How to Steal a Million', 1966, 'https://www.imdb.com/title/tt0060522/', 7.6, 22981, 0.379239022731781)
('P

In [91]:
z = Recommender('models/w2v_limitingfactor_v3.36.model')
z.predict(good_list, bad_list, hist_list, val_list, ratings_dict, n=100, harshness=1, rec_movies=False, scoring=True,)

The model recommended 28 movies that were on the watchlist!

		 Average Rating: 7.985106382978725



## Best model so far

LimitingFactor_v3.51 is the model to beat. It performs consistently well across various tests, which are detailed below.

In [67]:
s = Recommender('models/w2v_limitingfactor_v3.36.model')
s.predict(good_list, bad_list, hist_list, val_list, ratings_dict, n=100, harshness=1, rec_movies=False, scoring=True,)
# s.predict(aj2, n=100, harshness=1)

The model recommended 19 movies that were on the watchlist!

		 Average Rating: 8.156666666666666



### Add test cases

In [121]:
# Early test cases

# A list of some Coen Bros movies.
coen_bros = ['116282', '2042568', '1019452', 
             '1403865', '190590', '138524', 
             '335245', '477348', '887883', '101410']

# Data scientist's recent watches.
cooper_recent = ['0053285', '0038650', '0046022', 
                 '4520988', '1605783', '6751668', 
                 '0083791', '0115685', '0051459', 
                 '8772262', '0061184', '0041959',
                 '7775622']

# dirkh public letterboxd recent watches.
dirkh = ['7975244', '8106534', '1489887', 
         '1302006', '7286456', '6751668', 
         '8364368', '2283362', '6146586', 
         '2194499', '7131622', '6857112']

# Marvin watches
marvin = ['7286456', '0816692', '2543164', '2935510', 
          '2798920', '0468569', '5013056', '1375666', 
          '3659388', '0470752', '0266915', '0092675', 
          '0137523', '0133093', '1285016']  

# Gabe watches
gabe = ['6292852','0816692','2737304','3748528',
        '3065204','4154796','1536537','1825683',
        '1375666','8236336','2488496','1772341',
        '0317705','6857112','5052448']

# Eric watches
eric = ['2974050','1595842','0118539','0093405',
        '3216920','1256535','5612742','3120314',
        '1893371','0046248','0058548','0199481',
        '2296777','0071198','0077834']

chuckie = ['4263482',
'0084787',
'3286052',
'5715874',
'1172994',
'4805316',
'3139756',
'8772262',
'7784604',
'1034415',]

harlan = ['1065073','5052448','0470752','5688932','1853728','1596363','0432283','6412452','4633694','9495224','0443453','0063823',
          '0066921','0405296','1130884','1179933','0120630','0268126','0137523','0374900','8772262','0116996','0107290','7339248']

ryan = ['0166924','2866360','0050825','2798920','3416742','0060827','1817273','0338013','0482571','5715874','2316411','4550098']

karyn = ['4425200','0464141','1465522','0093779','0099810','0076759','3748528','6763664','0317740','2798920','0096283','0258463','0118799','0058092','0107290','0045152','0106364']

richard = ['0074119','0064115','0070735','0080474','0061512','0067774','0057115','0070511','0081283',
           '0065126','0068421','0078227','0079100','0078966','0081696','0082085','0072431','0075784',
           '0093640','0098051','0094226','0097576','0099810','0081633','0080761','0077975','0085244','0095159','0101969']

joe = ['6335734','0291350','0113568','0208502','0169858','0095327','0097814','0983213','0094625','7089878']

lena = ['1990314','3236120','1816518','0241527','0097757','0268978','0467406','2543164','2245084','3741834']

wade = ['0118665','0270846','0288441','2287250','2287238','8668804','9448868','1702443','1608290','5519340']

aj1 = ['0087995','0118694','0181689','0061184','0063032','2402927','4633694','0058946','0103074','0060196',
       '2543164','0109445','0245429','5105250','0088846','0370986','0246578','0053114','0014429','0047478',
       '0081505','2396224','0054215','1259521','0096283','0095159','0093779','0087544']

aj2 = ['0173716','0086541','0119809','0109445','0112887','0120879','0081455','0079813','0087995','0156610',
       '0097940','0089886','0088846','0090967','1523483','0109424','0102536','0105793','0246578','0370986']

# Plot my movies

In [6]:
!pip install umap

You should consider upgrading via the 'c:\users\cooper\anaconda3\python.exe -m pip install --upgrade pip' command.


In [6]:
import numpy as np
import umap
reducer = umap.UMAP(random_state=42)

In [87]:
s = Recommender('models/w2v_limitingfactor_v3.36.model')
len(s._get_model().wv.vocab.keys())

24691

In [12]:
all_movies = np.array(s._get_model().wv.vectors)
my_movies = np.array([s._get_model()[i] for i in hist_list+good_list+bad_list if i in s._get_model()])
histed = np.array([1 for i in hist_list if i in s._get_model()])
gooded = np.array([2 for i in good_list if i in s._get_model()])
badded = np.array([3 for i in bad_list if i in s._get_model()])

legend = np.concatenate((histed, gooded, badded))

In [13]:
all_movies.shape

(24691, 100)

In [14]:
legend.shape

(711,)

In [15]:
np.savetxt('all_movie_vectors.tsv', all_movies, delimiter="\t")

In [58]:
reducer = reducer.fit(all_movies)

In [61]:
embedding = reducer.transform(all_movies)

(9505, 13)

In [105]:
s = Recommender('models/w2v_limitingfactor_v3.36.model')
# s.id_book = s.id_book[s.id_book['num_votes']>8000]
clf = s._get_model()

id_list = s.id_book['id'].tolist()
title_list = s.id_book['primaryTitle'].tolist()
year_list = s.id_book['startYear'].tolist()
runtime_list = s.id_book['runtimeMinutes'].tolist()
genres_list = s.id_book['genres'].tolist()
rating_list = s.id_book['average_rating'].tolist()
votes_list = s.id_book['num_votes'].tolist()

included = []
metadata = [['title', 'year', 'avg_rating', '#votes', 'genres', 'runtime_min', 'id']]
for pos, i in enumerate(id_list):
    i = fill_id(i)
    if str(i) in clf.wv.vocab:
        included.append(clf[i])
        metadata.append([title_list[pos], year_list[pos], rating_list[pos], votes_list[pos], genres_list[pos], runtime_list[pos] ,fill_id(id_list[pos])])

included.append(z.my_vec)
metadata.append(['Coop Williams taste vector', '2020', '4', '389', 'action', '25', '0000000'])
        
all_movies = np.array(id_list)
all_metadata = np.array(metadata)
np.savetxt('vectors_all.tsv', included, delimiter="\t")
np.savetxt('metadata_all.tsv', all_metadata, delimiter="\t", fmt='%1s', encoding='utf-8')

In [106]:
print(len(all_metadata), len(included))

24692 24691


In [107]:
np.array(included).shape

(24691, 100)

In [ ]:
s.id_book.columns

In [ ]:
test = s._get_info(x) for x in 

In [73]:
!pip install bokeh

You should consider upgrading via the 'c:\users\cooper\anaconda3\python.exe -m pip install --upgrade pip' command.


In [77]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from bokeh.plotting import figure, show, output_file

# fig = px.scatter([1, 2]) 
# #             c=[sns.color_palette()[x] for x in legend])
# # plt.gca().set_aspect('equal', 'datalim')
# fig.show()

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

p.scatter(embedding[:, 0], embedding[:, 1], fill_alpha=0.6,
          line_color=None)

output_file("color_scatter.html", title="color_scatter.py example")

show(p)  # open a browser

# Score the models on various test cases

The below cell is your one-stop shop for scoring the various trained models and comparing their performance. Before using it, check the following:

    - The database connection is live (top cell of this notebook).
    
    - You have run the two cells under the headings "Define inferencing functions" and "prep generic data"
    
    - `id_book` is reading a CSV containing data from the IMDb title.basics.tsv document found [here](https://datasets.imdbws.com/).
    
    - `test_users` list contains the names of folders containing users' Letterboxd data that you want to use for scoring.
    
    - `model_list` contains the names of the versions you want to score.
    
Scoring is acheived using two metrics:
    1. Watchlist validation: How many movies were found that the user has already indicated interest in?
    2. Avg. Rating: What is the average rating of the movies recommended?
    
The cell below scores all models with these metrics and records the results in three dataframes. The first two dataframes, match_test_results and rating_test_results, use only different configurations of the 'cooper' data. This is meant to demonstrate various scenarios where more or less data are provided from the user, and different settings are selected. The third dataframe records scoring from both metrics across all users you care to test. For the sake of privacy, I have not included my test data with this repo, so future experimenters will have to gather their own Letterboxd data.

In [123]:
# log test results for 3 tests.

# df to lookup ids from titles
id_book = pd.read_csv('exported_data/title_basics_small.csv')

# define "user-test" test cases. set 'cooper' to last so it defines good_list, bad_list etc. for the cooper test cases
# each name must correspond to a folder of letterboxd data under "exported_data"
'''EXAMPLE file structure
current directory
    |
    thisnotebook.ipynb
    w2v_someversion.model
    ...
    |
    /exported_data
        |
        /eric
        ... other folders
        /cooper
            |
            ratings.csv
            watched.csv
            watchlist.csv
'''
test_users = ['eric', 'wade', 'aj', 'kelly', 'cooper', 'thomas'] 

# these names must match the file names of the versions you want to test, without prefix or extensions
model_list = ['mistakenot',
              'limitingfactor_v1', 'limitingfactor_v2', 
              'limitingfactor_v3', 'limitingfactor_v3.36', 'limitingfactor_v3.5', 'limitingfactor_v3.51', 'limitingfactor_v3.6', 
              'limitingfactor_v4', 'limitingfactor_v4.1', 'limitingfactor_v4.12']

###########################################################################################
# Nothing below this needs to be configured, unless you want to change the tests themselves.
###########################################################################################

# import user Letterboxd data
test_users_data = {}
for user in test_users:
    user_data = {}
    path = f"exported_data/letterboxd/{user}/"
    ratings = pd.read_csv(f'{path}ratings.csv')
    watched = pd.read_csv(f'{path}watched.csv')
    watchlist = pd.read_csv(f'{path}watchlist.csv')
    good_list, bad_list, hist_list, val_list, ratings_dict = prep_data(
                                    ratings, watched_df=watched, watchlist_df=watchlist, good_threshold=4, bad_threshold=3)
    user_data.update([("good_list", good_list), ("bad_list", bad_list), 
                      ("hist_list", hist_list), ("val_list", val_list), 
                      ("ratings_dict", ratings_dict)])
    test_users_data.update([(str(user), user_data)])

# empty dictionary to use when user elects not to give extra weight to extreme ratings
no_weights = {}

# define "cooper" test cases
params = [
    (cooper_recent, [], 1, ratings_dict), #1
    (cooper_recent, bad_list, 1, ratings_dict),
    (cooper_recent, bad_list, 2, ratings_dict),
    (cooper_recent, bad_list, 3, ratings_dict),
    (good_list, [], 1, ratings_dict), #5
    (good_list, bad_list, 1, ratings_dict),
    (good_list, bad_list, 2, ratings_dict),
    (good_list, bad_list, 3, ratings_dict),
    (cooper_recent, [], 1, no_weights),
    (cooper_recent, bad_list, 1, no_weights), #10
    (cooper_recent, bad_list, 2, no_weights),
    (cooper_recent, bad_list, 3, no_weights),
    (good_list, [], 1, no_weights),
    (good_list, bad_list, 1, no_weights),
    (good_list, bad_list, 2, no_weights), #15
    (good_list, bad_list, 3, no_weights),
#     (good_3p5_list, [], 1),
#     (good_3p5_list, bad_list, 1), #10
#     (good_3p5_list, bad_list, 2),
#     (good_3p5_list, bad_list, 3),
#     (cooper_recent, karyn, 1),
#     (good_list, karyn, 1),
#     (good_3p5_list, karyn, 1), #15
]

mods = dict(zip(model_list, ["models/w2v_" + x + ".model" for x in model_list])) # zip model paths with their names

# attributes/training hyperparameters we want to tabulate
attr_list = ['vector_size', 'corpus_count', 'corpus_total_words', 
             'window', 'sg', 'hs', 'negative', 
             'alpha', 'min_alpha', 
             'sample', 'epochs']

# the three tests we will run
match_test_results = pd.DataFrame({'model':model_list}) # Test different settings for "cooper", score on watchlist items found
rating_test_results = pd.DataFrame({'model':model_list}) # Test different settings for "cooper", score on avg. rating of recs
user_test_results = pd.DataFrame({'model':model_list}) # Test different users on the same settings, score on watchlist items found


# Get model training parameters for models
for attr in attr_list:
    match_test_results[str(attr)] = match_test_results['model'].apply(lambda x: getattr(Recommender(mods[x])._get_model(), attr))
    rating_test_results[str(attr)] = match_test_results[str(attr)]

# add ns_exponent parameter, which is buried in the model.vocabulary (can't be gotten with getattr())
for df in [match_test_results, rating_test_results]:
    df['ns_exponent'] = df['model'].apply(lambda x: Recommender(mods[x])._get_model().vocabulary.ns_exponent)

# get match scores, average ratings, attributes for all "cooper" test cases
count = 0
for i, j, k, m in params:
        count+=1
        print(count, "\t")
        match_test_results[str(count)] = match_test_results['model'].apply(
                                                lambda x: Recommender(mods[x]).predict(input=i, bad_movies=j, 
                                                                                          val_list=val_list, ratings_dict=m, 
                                                                                          n=100, harshness=k, 
                                                                                          rec_movies=False, show_vibes=False, 
                                                                                          scoring=True, return_scores=True))
        rating_test_results[str(count)] = match_test_results[str(count)].apply(lambda x: x[1])
        match_test_results[str(count)] = match_test_results[str(count)].apply(lambda x: x[0])
        
for user, data in test_users_data.items():
    print(str(user))
    user_test_results[str(user)] = user_test_results['model'].apply(
                                                lambda x: Recommender(mods[x]).predict(input=data['good_list'], 
                                                                                        bad_movies=data['bad_list'], 
                                                                                        val_list=data['val_list'], 
                                                                                        ratings_dict=data['ratings_dict'], 
                                                                                        n=100, harshness=1, 
                                                                                        rec_movies=False, show_vibes=False, 
                                                                                        scoring=True, return_scores=True))
        

1 	
2 	
3 	
4 	
5 	
6 	
7 	
8 	
9 	
10 	
11 	
12 	
13 	
14 	
15 	
16 	
eric
wade
aj
kelly
cooper
thomas


## Test cases legend:

```
1 	(cooper_recent, [], 1, ratings_dict)

2 	(cooper_recent, bad_list, 1, ratings_dict)

3 	(cooper_recent, bad_list, 2, ratings_dict)

4 	(cooper_recent, bad_list, 3, ratings_dict)

5 	(good_list, [], 1, ratings_dict)

6 	(good_list, bad_list, 1, ratings_dict) 
        # hardcore movie fan looking for something crazy

7 	(good_list, bad_list, 2, ratings_dict)

8 	(good_list, bad_list, 3, ratings_dict)

9 	(cooper_recent, [], 1, no_weights) 
        # small list of movies with only a few good movie ratings

10 	(cooper_recent, bad_list, 1, no_weights) 
        # small list of movies with some bad movies too

11 	(cooper_recent, bad_list, 2, no_weights)

12 	(cooper_recent, bad_list, 3, no_weights)

13 	(good_list, [], 1, no_weights) 
        # Unlikely case

14 	(good_list, bad_list, 1, no_weights) 
        # Hardcore movie fan

15 	(good_list, bad_list, 2, no_weights)

16 	(good_list, bad_list, 3, no_weights)
```

### Show test results

Overall, the best models scored seem to be limitingfactor_v1 and limitingfactor_v3.51. Unfortunately I did have the presence of mind to document the settings for the former's training data. But I believe it may have been `m=7, n=10`.

In [124]:
pd.set_option('display.max_columns', 50)
match_test_results # for each model: training params, validation score out of 100 for 16 test cases

,model,vector_size,corpus_count,corpus_total_words,window,sg,hs,negative,alpha,min_alpha,sample,epochs,ns_exponent,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,mistakenot,100,17812,904140,10,1,0,10,0.03,0.0007,0.0010,10,0.75,5,9,8,6,5,11,19,10,5,11,5,4,4,13,18,9
1,limitingfactor_v1,100,17812,817687,10,1,0,10,0.03,0.0007,0.0001,90,0.50,13,14,13,14,18,21,29,26,16,20,22,17,18,19,28,26
2,limitingfactor_v2,100,13641,766970,10,1,0,10,0.03,0.0007,0.0001,90,0.50,15,17,14,14,14,21,28,23,13,19,20,17,13,19,27,22
3,limitingfactor_v3,100,35371,933359,10,1,0,10,0.03,0.0007,0.0001,90,0.50,10,13,14,13,18,26,32,29,20,21,23,24,17,25,31,28
4,limitingfactor_v3.36,100,35371,933359,10,1,0,10,0.03,0.0007,0.0001,1500,0.30,13,16,14,12,19,35,31,28,19,16,20,21,17,35,33,28
5,limitingfactor_v3.5,100,35371,933359,10,1,0,10,0.03,0.0007,0.0001,90,0.35,6,12,10,10,20,27,30,28,17,21,25,20,18,24,31,28
6,limitingfactor_v3.51,100,35371,933359,10,1,0,10,0.03,0.0007,0.0001,500,0.35,17,17,17,17,17,32,32,26,19,20,27,27,16,33,32,25
7,limitingfactor_v3.6,100,44438,986835,10,1,0,10,0.03,0.0007,0.0001,1500,0.30,13,11,12,14,27,47,44,41,20,20,34,35,27,48,43,41
8,limitingfactor_v4,300,35371,933359,10,1,0,10,0.03,0.0007,0.0001,30,0.50,2,7,4,3,3,5,9,5,1,2,2,2,3,5,8,5
9,limitingfactor_v4.1,300,35371,933359,10,1,0,10,0.03,0.0007,0.0001,100,0.50,6,11,8,9,3,19,14,8,3,12,6,5,3,21,12,8


In [125]:
rating_test_results # for each model: training params, average rating of movies recommended for 16 test cases

,model,vector_size,corpus_count,corpus_total_words,window,sg,hs,negative,alpha,min_alpha,sample,epochs,ns_exponent,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,mistakenot,100,17812,904140,10,1,0,10,0.03,0.0007,0.0010,10,0.75,7.011224,7.272165,7.114433,7.092784,6.901124,7.245918,7.402410,6.954321,7.267010,7.111111,7.225773,7.240625,6.906818,7.255102,7.397590,7.011111
1,limitingfactor_v1,100,17812,817687,10,1,0,10,0.03,0.0007,0.0001,90,0.50,7.468041,7.365306,7.410417,7.416667,8.065574,7.453571,8.141667,8.250000,7.512903,7.411458,7.579570,7.521739,8.066129,7.429213,8.137288,8.246296
2,limitingfactor_v2,100,13641,766970,10,1,0,10,0.03,0.0007,0.0001,90,0.50,7.443299,7.403061,7.365625,7.361458,7.987097,7.512791,8.106897,8.175000,7.427957,7.445833,7.507447,7.452174,7.936508,7.468132,8.098276,8.174510
3,limitingfactor_v3,100,35371,933359,10,1,0,10,0.03,0.0007,0.0001,90,0.50,7.435714,7.303061,7.395876,7.383505,8.227273,7.562500,8.268966,8.386792,7.554348,7.306383,7.509677,7.595699,8.189286,7.551648,8.255932,8.339623
4,limitingfactor_v3.36,100,35371,933359,10,1,0,10,0.03,0.0007,0.0001,1500,0.30,7.441237,7.351546,7.308333,7.325000,8.222414,7.767500,8.316364,8.301852,7.652128,7.293617,7.582796,7.575532,8.148276,7.711628,8.298214,8.320000
5,limitingfactor_v3.5,100,35371,933359,10,1,0,10,0.03,0.0007,0.0001,90,0.35,7.466327,7.377778,7.360825,7.400000,8.190164,7.559036,8.287931,8.325926,7.651064,7.286458,7.575269,7.575269,8.173333,7.541111,8.246552,8.320755
6,limitingfactor_v3.51,100,35371,933359,10,1,0,10,0.03,0.0007,0.0001,500,0.35,7.509184,7.389691,7.360825,7.355208,7.931148,7.727160,8.172414,8.001724,7.582796,7.337895,7.644086,7.611957,7.862903,7.702326,8.177586,7.963793
7,limitingfactor_v3.6,100,44438,986835,10,1,0,10,0.03,0.0007,0.0001,1500,0.30,7.101020,7.320408,7.312245,7.293814,8.351724,8.074667,8.295455,8.327419,7.592473,7.532979,7.747872,7.770968,8.351724,8.041772,8.279104,8.329032
8,limitingfactor_v4,300,35371,933359,10,1,0,10,0.03,0.0007,0.0001,30,0.50,7.264646,7.122222,7.274490,7.342857,6.915464,7.119792,7.429348,7.180000,6.812121,6.647312,6.771277,6.627083,6.853608,7.138144,7.407778,7.149474
9,limitingfactor_v4.1,300,35371,933359,10,1,0,10,0.03,0.0007,0.0001,100,0.50,7.292857,7.184694,7.282653,7.303061,6.713830,7.515476,7.019048,6.835632,6.558696,7.260000,6.797778,6.534444,6.694624,7.516092,6.939286,6.860920


In [126]:
user_test_results # for each model: validation score and avg. rating for a user's inputs.

,model,eric,wade,aj,kelly,cooper,thomas
0,mistakenot,"(1, 7.299999999999996)","(4, 7.682474226804127)","(6, 7.234375000000001)","(8, 7.558064516129031)","(22, 7.633333333333332)","(11, 7.245918367346941)"
1,limitingfactor_v1,"(3, 7.618888888888891)","(6, 7.460465116279073)","(3, 7.043333333333335)","(11, 7.730232558139535)","(21, 7.665151515151515)","(21, 7.45357142857143)"
2,limitingfactor_v2,"(1, 7.59186046511628)","(10, 7.539325842696628)","(6, 7.212765957446808)","(12, 7.739772727272731)","(21, 7.667187500000001)","(21, 7.5127906976744185)"
3,limitingfactor_v3,"(4, 7.673493975903615)","(11, 7.539759036144578)","(5, 7.288043478260868)","(16, 7.817857142857145)","(22, 7.807272727272728)","(26, 7.5625)"
4,limitingfactor_v3.36,"(4, 7.841772151898734)","(17, 7.604225352112677)","(6, 7.496666666666672)","(20, 7.980000000000001)","(28, 7.989795918367348)","(35, 7.7675)"
5,limitingfactor_v3.5,"(4, 7.746428571428571)","(10, 7.508045977011496)","(6, 7.327472527472528)","(14, 7.845000000000001)","(22, 7.784210526315789)","(27, 7.559036144578312)"
6,limitingfactor_v3.51,"(5, 7.801219512195122)","(15, 7.611538461538464)","(6, 7.559340659340662)","(16, 7.940789473684211)","(25, 7.8999999999999995)","(32, 7.727160493827163)"
7,limitingfactor_v3.6,"(5, 8.104615384615384)","(17, 7.642253521126764)","(19, 8.032499999999997)","(22, 8.120000000000003)","(19, 8.171875)","(47, 8.074666666666669)"
8,limitingfactor_v4,"(0, 6.976842105263159)","(5, 7.422222222222221)","(1, 6.941935483870969)","(3, 7.352127659574468)","(2, 7.329032258064515)","(5, 7.119791666666667)"
9,limitingfactor_v4.1,"(0, 7.341025641025642)","(8, 7.38875)","(4, 6.950000000000001)","(9, 7.545454545454544)","(9, 7.366129032258064)","(19, 7.5154761904761855)"


In [127]:
# # Export test results
match_test_results.to_csv("w2v_match_test_results.csv", index=False)
rating_test_results.to_csv("w2v_rating_test_results.csv", index=False)
user_test_results.to_csv("w2v_user_test_results.csv", index=False)

# Beyond this point: Recommendation examples (long output)

## Best settings for LimitingFactor_v1

In [38]:
# focus on the best settings for LimitingFactor_v1
params = [
    (cooper_recent, bad_list, 2), #3
    (cooper_recent, bad_list, 3), #4
    (good_list, bad_list, 1), #6
    (good_3p5_list, bad_list, 1), #10
    (good_list, karyn, 1), #14
    (good_3p5_list, karyn, 1), #15
]

count = 0
settings = [3, 4, 6, 10, 14, 15]
# Test all the promising parameter sets
for i, j, k in params:
    print(settings[count], "\t")
    prediction = s.predict(input=i, bad_movies=j, n=100, harshness=k, rec_movies=True, show_vibes=False, scoring=True)
    for i in prediction:
        print(f"{i[0]}\t{i[1]}\n\t{i[2]}\n\tAvg. Rating: {i[3]}\n\t# Votes: {i[4]}\n\tSimilarity: {i[5]}\n\n")
    count+=1

"""
Settings graded out of 100:
3... 91
4... 86
6... 89 (lots of old stuff)
10.. 90
14.. 91
15.. 90

I'm inclined to pronounce model LimitingFactor_v1 a success, because it gives 
diverse and high-quality results in more cases than its predecessor.
In 6 cases it gives 20+ watchlisted movies, where before we only got 3.

In other words, it gives good results in cases where input data may be rich or sparse.

"""


3 	
The model recommended 23 movies that were on the watchlist!

		 Average Rating: 7.444999999999996

Marriage Story	2019
	https://www.imdb.com/title/tt7653254/
	Avg. Rating: 8.1
	# Votes: 130590
	Similarity: 0.6401818990707397


Joker	2019
	https://www.imdb.com/title/tt7286456/
	Avg. Rating: 8.6
	# Votes: 620299
	Similarity: 0.6330293416976929


The Lighthouse	2019
	https://www.imdb.com/title/tt7984734/
	Avg. Rating: 7.8
	# Votes: 52091
	Similarity: 0.6307841539382935


Apollo 11	2019
	https://www.imdb.com/title/tt8760684/
	Avg. Rating: 8.2
	# Votes: 15085
	Similarity: 0.6201057434082031


Once Upon a Time... in Hollywood	2019
	https://www.imdb.com/title/tt7131622/
	Avg. Rating: 7.8
	# Votes: 342653
	Similarity: 0.6006509065628052


Knives Out	2019
	https://www.imdb.com/title/tt8946378/
	Avg. Rating: 8.0
	# Votes: 124017
	Similarity: 0.6003093719482422


A Star Is Born	2018
	https://www.imdb.com/title/tt1517451/
	Avg. Rating: 7.7
	# Votes: 293127
	Similarity: 0.5982445478439331


The

### Compare two test cases for v1

In [32]:
#Compare the unique recs of settings 7 and 11 from above.
#The param difference is that setting 7 uses movies rated 4 and above,
#while setting 11 uses movies rated 3.5 and above.

p4 = s.predict(input=good_list, bad_movies=bad_list, n=200, harshness=2, scoring=True)
p35 = s.predict(input=good_3p5_list, bad_movies=bad_list, n=200, harshness=2, scoring=True)

print(len(p4), len(p35))

p4_unique = [x for x in p4 if x[0] not in [x[0] for x in p35]]
p35_unique = [x for x in p35 if x[0] not in [x[0] for x in p4]]

print(len(p4_unique), len(p35_unique))


The model recommended 29 movies that were on the watchlist!

		 Average Rating: 7.786516853932587

The model recommended 27 movies that were on the watchlist!

		 Average Rating: 7.732142857142859

89 84
27 22


In [33]:
for i in p4_unique:
    print(f"{i[0]}\t{i[1]}\n\t{i[2]}")

Le parfum d'Yvonne	1994
	https://www.imdb.com/title/tt0110776/
Beautiful	1951
	https://www.imdb.com/title/tt0043332/
Journey to Italy	1954
	https://www.imdb.com/title/tt0046511/
A Story from Chikamatsu	1954
	https://www.imdb.com/title/tt0046851/
Manhattan	1979
	https://www.imdb.com/title/tt0079522/
The Circus	1928
	https://www.imdb.com/title/tt0018773/
Cold War	2018
	https://www.imdb.com/title/tt6543652/
Aguirre, the Wrath of God	1972
	https://www.imdb.com/title/tt0068182/
La Dolce Vita	1960
	https://www.imdb.com/title/tt0053779/
The Thin Blue Line	1988
	https://www.imdb.com/title/tt0096257/
Lancelot of the Lake	1974
	https://www.imdb.com/title/tt0071737/
Conversation Piece	1974
	https://www.imdb.com/title/tt0071585/
4 Months, 3 Weeks and 2 Days	2007
	https://www.imdb.com/title/tt1032846/
Romero	1989
	https://www.imdb.com/title/tt0098219/
Stagecoach	1939
	https://www.imdb.com/title/tt0031971/
Miller's Crossing	1990
	https://www.imdb.com/title/tt0100150/
Ankur: The Seedling	1974
	https:

In [212]:
for i in p35_unique:
    print(f"{i[0]}\t{i[1]}\n\t{i[2]}")

Once Upon a Time in High School: The Spirit of Jeet Kune Do	2004
	https://www.imdb.com/title/tt0390205/
Chillar Party	2011
	https://www.imdb.com/title/tt1841542/
Angel's Egg	1985
	https://www.imdb.com/title/tt0208502/
Unacknowledged	2017
	https://www.imdb.com/title/tt6400614/
Lootera	2013
	https://www.imdb.com/title/tt2224317/
Awe!	2018
	https://www.imdb.com/title/tt7797658/
Vada Chennai	2018
	https://www.imdb.com/title/tt5959980/
Aruvi	2016
	https://www.imdb.com/title/tt5867800/
Big Fish & Begonia	2016
	https://www.imdb.com/title/tt1920885/
Thithi	2015
	https://www.imdb.com/title/tt4881362/
Boy and the World	2013
	https://www.imdb.com/title/tt3183630/
Neon Genesis Evangelion: The End of Evangelion	1997
	https://www.imdb.com/title/tt0169858/
That Girl in Yellow Boots	2010
	https://www.imdb.com/title/tt1580704/
The Breath	2009
	https://www.imdb.com/title/tt1171701/
The Stupids	1996
	https://www.imdb.com/title/tt0117768/
To the Wonder	2012
	https://www.imdb.com/title/tt1595656/
The Meyer

### Examine ideal case

In [34]:
prediction = s.predict(input=good_list, bad_movies=bad_list, n=200, harshness=2)
for i in prediction:
    print(f"{i[0]}\t{i[1]}\n\t{i[2]}\n\tAvg. Rating: {i[3]}\n\t# Votes: {i[4]}\n\tSimilarity: {i[5]}\n\n")

One Flew Over the Cuckoo's Nest	1975
	https://www.imdb.com/title/tt0073486/
	Avg. Rating: 8.7
	# Votes: 860775
	Similarity: 0.637021541595459


The Devil, Probably	1977
	https://www.imdb.com/title/tt0075938/
	Avg. Rating: 7.3
	# Votes: 3150
	Similarity: 0.6019104719161987


Do the Right Thing	1989
	https://www.imdb.com/title/tt0097216/
	Avg. Rating: 7.9
	# Votes: 78577
	Similarity: 0.5995447635650635


Chinatown	1974
	https://www.imdb.com/title/tt0071315/
	Avg. Rating: 8.2
	# Votes: 275306
	Similarity: 0.5974133014678955


Sunset Blvd.	1950
	https://www.imdb.com/title/tt0043014/
	Avg. Rating: 8.4
	# Votes: 186742
	Similarity: 0.5961176753044128


On the Waterfront	1954
	https://www.imdb.com/title/tt0047296/
	Avg. Rating: 8.1
	# Votes: 132973
	Similarity: 0.5904337763786316


La Collectionneuse	1967
	https://www.imdb.com/title/tt0061495/
	Avg. Rating: 7.5
	# Votes: 5523
	Similarity: 0.5855226516723633


The Elephant Man	1980
	https://www.imdb.com/title/tt0080678/
	Avg. Rating: 8.1
	# Vo

## Best settings for v2

In [33]:
"""LimitingFactor_v2 is trained on movies rated 8 stars and above. The first thing I notice 
about it is that, using my movie history, whenever its recommendations get a high
average score, it strays farther from my watchlist. So this is where it may become 
challenging to evaluate the model: are we optimizing for stuff I haven't heard of, or average rating?"""

# focus on the best settings for LimitingFactor_v2
params = [
    (good_list, bad_list, 3), #8
    (good_3p5_list, bad_list, 2), #11
    (good_3p5_list, bad_list, 3)  #12
]

count = 0
settings = [8, 11, 12]
# Test all the promising parameter sets
for i, j, k in params:
    print(settings[count], "\t")
    prediction = s.predict(input=i, bad_movies=j, n=100, harshness=k, rec_movies=True, show_vibes=False, scoring=True)
    for i in prediction:
        print(f"{i[0]}\t{i[1]}\n\t{i[2]}\n\tAvg. Rating: {i[3]}\n\t# Votes: {i[4]}\n\tSimilarity: {i[5]}\n\n")
    count+=1

"""
Wow. Model v2 is crazily overfitted. Which means that it returns so many duplicates that it recommends nothing new.
This technically shouldn't be happening, because we are actively removing duplicates. But _remove_dupes() cannot,
as of this time, remove titles that don't have ILIKE matches in the database. At n=25, we get a div0 error because
the recs list is empty. So at n=30 we're getting just the unhandled dupes as a result.

Conclusions:
1. v2 is overfitted.
2. Average Rating is not a good metric to use for v2.
3. v2 fails the watchlist test spectacularly due to (1).

As a result, we should train on ratings 7-10.

"""


8 	
The model recommended 7 movies that were on the watchlist!

		 Average Rating: 7.970000000000001

Schindler's List	1993
	https://www.imdb.com/title/tt0108052/
	Avg. Rating: 8.9
	# Votes: 1138968
	Similarity: 0.6192294359207153


One Flew Over the Cuckoo's Nest	1975
	https://www.imdb.com/title/tt0073486/
	Avg. Rating: 8.7
	# Votes: 860775
	Similarity: 0.6185670495033264


The Last Supper	1976
	https://www.imdb.com/title/tt0075363/
	Avg. Rating: 7.5
	# Votes: 518
	Similarity: 0.6030973196029663


Pan's Labyrinth	2006
	https://www.imdb.com/title/tt0457430/
	Avg. Rating: 8.2
	# Votes: 588428
	Similarity: 0.5942332744598389


Casino	1995
	https://www.imdb.com/title/tt0112641/
	Avg. Rating: 8.2
	# Votes: 432378
	Similarity: 0.5923681855201721


78/52: Hitchcock's Shower Scene	2017
	https://www.imdb.com/title/tt4372240/
	Avg. Rating: 7.3
	# Votes: 2652
	Similarity: 0.5904847383499146


Chinatown	1974
	https://www.imdb.com/title/tt0071315/
	Avg. Rating: 8.2
	# Votes: 275306
	Similarity: 0.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:60: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).


"\nWow. Model v2 is crazily overfitted. Which means that it returns so many duplicates that it recommends nothing new.\nThis technically shouldn't be happening, because we are actively removing duplicates. But _remove_dupes() cannot,\nas of this time, remove titles that don't have ILIKE matches in the database. At n=25, we get a div0 error because\nthe recs list is empty. So at n=30 we're getting just the unhandled dupes as a result.\n\nConclusions:\nv2 is overfitted.\nAverage Rating is not a good metric to use for v2.\nv2 fails the \n\n\n\n\n"

# Junkyard

In [ ]:
# s = Recommender('w2v_limitingfactor_v3.51.model')


# possible good inputs: cooper_recent (13 most recent), good_list(my 4-5 star ratings), good_3p5_list (my 3.5-5 star ratings)
# possible bad_movies: [] (not removing dislikes), bad_list(my ratings 3 and below), karyn (individual whose taste markedly differs from mine)
# possible harshness: 1, 2, 3 (harshness 1 strongly removes disliked movies, while harshness 3 removes with a third of the strength)
# define test cases
# params = [
#     (cooper_recent, [], 1), #1
#     (cooper_recent, bad_list),
#     (cooper_recent, bad_list),
#     (cooper_recent, bad_list),
#     (good_list, [], 1), #5
#     (good_list, bad_list, 1),
#     (good_list, bad_list, 2),
#     (good_list, bad_list, 3),
#     (good_3p5_list, [], 1),
#     (good_3p5_list, bad_list, 1), #10
#     (good_3p5_list, bad_list, 2),
#     (good_3p5_list, bad_list, 3),
#     (cooper_recent, karyn, 1),
#     (good_list, karyn, 1),
#     (good_3p5_list, karyn, 1), #15
# ]
# Good performance for v2MistakeNot: 6, 7, 11
# Good performance for v3LimitingFactor: 3, 4, 6, 10, 14, 15
# Good performance for v3LimitingFactor_3.51: 2, 3, 4, 6, 14
# Setting 6 seems robust to significant hyperparameter changes.